In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import time
from google.colab import drive

# 랜덤 User-Agent 리스트
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 17_2 like Mac OS X) AppleWebKit/537.36 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0"
]

def job_info_search(job_name, pages=10):
    base_url = "https://www.saramin.co.kr"
    data_list = []  # 전체 데이터 저장용 리스트
    visited_companies = set()  # 이미 방문한 기업을 저장할 집합
    session = requests.Session()  # 세션 유지

    for page in range(1, pages + 1):
        search_url = f"{base_url}/zf_user/search/recruit?search_area=main&search_done=y&search_optional_item=n&searchType=search&searchword={job_name}&recruitPage={page}&recruitSort=relation&recruitPageCount=40"

        # 요청 보내기
        headers = {"User-Agent": random.choice(user_agents)}
        response = session.get(search_url, headers=headers)

        if response.status_code != 200:
            print(f"❌ [Error] 메인 페이지 응답 코드: {response.status_code}")
            continue

        print(f"✅ 페이지 {page} 응답 코드:", response.status_code)

        # HTML 파싱
        soup = BeautifulSoup(response.text, 'html.parser')

        def safe_select(soup, selector):
            """ 선택한 요소가 있으면 텍스트를 반환하고, 없으면 None 반환 """
            element = soup.select_one(selector)
            return element.get_text(strip=True) if element else None

        for tag in soup.select('.corp_name'):
            a_tag = tag.find('a')
            if a_tag:
                company_name = a_tag.get_text(strip=True)  # 기업 이름 추출

                # 이미 방문한 기업이면 건너뛰기
                if company_name in visited_companies:
                    continue

                # 기업 이름을 방문한 기업 목록에 추가
                visited_companies.add(company_name)

                tag_name = a_tag['href']
                sub_url = f"{base_url}{tag_name}"
                sub_url_inner = sub_url.replace("view", "view-inner-salary")

                # 랜덤 대기 (IP 차단 방지) → 기존보다 약간 더 길게 설정
                time.sleep(random.uniform(5, 10))

                # 서브 페이지 요청
                headers = {"User-Agent": random.choice(user_agents)}
                response_sub = session.get(sub_url, headers=headers)

                # 🔹 기업 정보가 없는 경우 건너뛰기
                error_msg = safe_select(BeautifulSoup(response_sub.text, 'html.parser'), ".result_txt")
                if error_msg and "죄송합니다" in error_msg:
                    data_list.append([company_name, None, None, None, None, None, None, None, None])
                    print(f"⚠️ {sub_url} - 해당하는 기업정보 조회불가")
                    continue

                sub_soup = BeautifulSoup(response_sub.text, 'html.parser')

                response_inner = session.get(sub_url_inner, headers=headers)
                if response_inner.status_code != 200:
                    print(f"❌ [Error] {sub_url_inner} 응답 코드: {response_inner.status_code}")
                    continue

                sub_soup_inner = BeautifulSoup(response_inner.text, 'html.parser')

                # 기본값 할당 (오류 방지)
                name1 = name2 = desc1 = name4 = address = None
                min_sal = max_sal = avg_sal = None

                # 회사 정보 추출
                company_info = sub_soup.select_one('div.area_company_infos')
                if company_info:
                    # .company_summary_tit에서 최대 2개 추출
                    titles = [t.text.strip() for t in company_info.select('.company_summary_tit')[:2]]
                    name1 = titles[0] if len(titles) > 0 else None
                    name2 = titles[1] if len(titles) > 1 else None

                    # .company_summary_desc에서 최대 1개 추출
                    descs = [d.text.strip() for d in company_info.select('.company_summary_desc')[:1]]
                    desc1 = descs[0] if len(descs) > 0 else None

                    # dd.desc에서 최대 1개 추출 (name4)
                    desc_tags = [d.text.strip() for d in company_info.select('dd.desc')[:1]]
                    name4 = desc_tags[0] if len(desc_tags) > 0 else None

                    # p.ellipsis에서 마지막 요소 추출 (address)
                    address_tags = [p.text.strip() for p in company_info.select('p.ellipsis')]
                    address = address_tags[-1] if len(address_tags) > 0 else None

                # 추가 데이터 추출 (급여 정보)
                min_sal = safe_select(sub_soup_inner, ".min_txt")
                max_sal = safe_select(sub_soup_inner, ".max_txt")
                avg_sal = safe_select(sub_soup_inner, ".average_currency")

                data_list.append([company_name, name1, name2, desc1, name4, address, min_sal, max_sal, avg_sal])

    # 데이터프레임 생성
    df = pd.DataFrame(data_list, columns=["company_name", "name1", "name2", "desc1", "name4", "address", "min_sal", "max_sal", "avg_sal"])

    # 구글 드라이브 연결
    drive.mount('/content/drive')

    # 파일 저장 경로 지정
    file_path = f'/content/drive/MyDrive/KDT/크롤링_프로젝트/{job_name}_info2_desktop.csv'

    # 구글 드라이브에 저장
    df.to_csv(file_path, index=False, encoding="utf-8-sig")
    print(f"파일이 구글 드라이브에 저장되었습니다: {file_path}")

    print("✅ 크롤링 완료! CSV 저장됨.")

# 함수 실행
job_info_search('데이터분석가', 100)

✅ 페이지 1 응답 코드: 200
✅ 페이지 2 응답 코드: 200
✅ 페이지 3 응답 코드: 200
✅ 페이지 4 응답 코드: 200
✅ 페이지 5 응답 코드: 200
⚠️ https://www.saramin.co.kr/zf_user/company-info/view?csn=alVVR0tWNERjcUpDYWtNdVlIc2oxZz09 - 해당하는 기업정보 조회불가
⚠️ https://www.saramin.co.kr/zf_user/company-info/view?csn=QmRwcXZZNk5sNXgyNUFSWnFTaDV3Zz09 - 해당하는 기업정보 조회불가
⚠️ https://www.saramin.co.kr/zf_user/company-info/view?csn=Vi85WEZydEpUd0cxNGhvNkZpUitoQT09 - 해당하는 기업정보 조회불가
✅ 페이지 6 응답 코드: 200
⚠️ https://www.saramin.co.kr/zf_user/company-info/view?csn=ejBYRE5RU0pEemZ4M2dNVWZWUHpBUT09 - 해당하는 기업정보 조회불가
✅ 페이지 7 응답 코드: 200
⚠️ https://www.saramin.co.kr/zf_user/company-info/view?csn=VWdhL2tORmhCaWxVU1hBVG1NRWZLdz09 - 해당하는 기업정보 조회불가
✅ 페이지 8 응답 코드: 200
⚠️ https://www.saramin.co.kr/zf_user/company-info/view?csn=aUZzSWg2dFh4R3dlWGdVbnJqT0tkQT09 - 해당하는 기업정보 조회불가
⚠️ https://www.saramin.co.kr/zf_user/company-info/view?csn=V3VsUFkyY2F4bExZMkdUbnQrME5sUT09 - 해당하는 기업정보 조회불가
✅ 페이지 9 응답 코드: 200
✅ 페이지 10 응답 코드: 200
⚠️ https://www.saramin.co.kr/zf_user/company-i